In [96]:
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os
from time import sleep
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [97]:
game_url = 'http://www.snookerdatabase.co.uk/PlayerDetails.aspx?PlayerKey={player_key}&tourns=All&plStat=2&pg={page_num}#PlayerResults'
table_x_path = '//*[@id="Table1"]/tbody/tr[4]/td[2]/center/table[3]'
name_x_path = '//*[@id="Table1"]/tbody/tr[4]/td[2]/center/center[1]/b/font'

In [104]:
def create_driver(headless=False):
    # instantiate a chrome options object so you can set the size and headless preference"
    

In [105]:
def extract_table(x_path, header=False):
    table = driver.find_element_by_xpath(x_path)
    df = pd.read_html('<table>' +table.get_attribute('innerHTML') + '</table')[0]
    if header:
        df.columns = df.iloc[0]
        df = df[1:]
    return df

In [117]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--window-size=1920x1080')
driver = webdriver.Chrome(options=chrome_options)
combined_df = pd.DataFrame()
error_keys = []
for player_key in range(1,3200):
    for page_num in range(1,100):
        try:
            driver.get(game_url.format(player_key=player_key, page_num=page_num))
            name = driver.find_element_by_xpath(name_x_path).text
            results_df = extract_table(x_path=table_x_path, header=True)
            results_df['Name'] = name
            results_df['player_key'] = player_key
            assert results_df.shape[0] > 0
            combined_df = combined_df.append(results_df,sort=False)
        except Exception as e:
            if page_num == 1: 
                error_keys.append(player_key)
                print(e)
                print(f'failure {player_key}')
                try_flag = False

            else:
                print(f'player {name} key {player_key} - pages {page_num-1}')
            break

driver.quit()

player Steve Davis key 1 - pages 21
player Stephen Hendry key 2 - pages 18
player Jimmy White key 3 - pages 22
player John Parrott key 4 - pages 11
player John Higgins key 5 - pages 20
player Mark Selby key 6 - pages 16
player Stephen Maguire key 7 - pages 14
player Shaun Murphy key 8 - pages 16
player Anthony Hamilton key 9 - pages 12
player Ronnie O Sullivan key 10 - pages 18
player Matthew Stevens key 11 - pages 13
player Ali Carter key 12 - pages 11
player Ian McCulloch key 13 - pages 5
player Joe Swail key 14 - pages 11
player Fergal O Brien key 15 - pages 12
player Neil Robertson key 16 - pages 14
player Mark Allen key 17 - pages 12
player Peter Ebdon key 18 - pages 15
player Graeme Dott key 19 - pages 14
player Marco Fu key 20 - pages 14
player Joe Perry key 21 - pages 15
player Mark Williams key 22 - pages 20
player Michael Holt key 23 - pages 12
player Barry Hawkins key 24 - pages 14
player Ryan Day key 25 - pages 13
player Ding Junhui key 26 - pages 13
player Ken Doherty key 

SyntaxError: 'return' outside function (<ipython-input-117-cc204ac1c8e1>, line 31)

In [120]:
combined_df['date'] = pd.to_datetime(combined_df.Date,dayfirst=True)

In [121]:
combined_df['round_count'] = combined_df.groupby(['Tournament','date','Round'])['Result'].transform('count')*-1

In [122]:
combined_df.sort_values(by=['date','round_count'],inplace=True)

In [123]:
scores =combined_df.Score.str.split(' V ')

In [124]:
def parse_scores(scores,i):
    try:
        return int(scores[i])
    except:
        return np.nan

dedup

In [125]:
combined_df['first_score'] = scores.apply(lambda x: parse_scores(x,0))
combined_df['second_score'] = scores.apply(lambda x: parse_scores(x,1))


In [126]:
combined_df['highest_score'] = np.maximum(combined_df.first_score, combined_df.second_score)
combined_df['lowest_score'] = np.minimum(combined_df.first_score, combined_df.second_score)

In [127]:
combined_df['highest_name'] = np.maximum(combined_df.Name, combined_df.Opponent)
combined_df['lowest_name'] = np.minimum(combined_df.Name, combined_df.Opponent)


In [128]:
dedup_df = combined_df.drop_duplicates(subset=['highest_name','lowest_name','highest_score','lowest_score',
                                              'Tournament','Date','Round'])

In [129]:
combined_df.drop(columns = ['highest_name','lowest_name','highest_score','lowest_score'],inplace=True)
dedup_df.drop(columns = ['highest_name','lowest_name','highest_score','lowest_score'],inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [130]:
combined_df.round_count *= -1
dedup_df.round_count *= -1

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [131]:
combined_df.to_csv('data/snooker_outcomes_all.csv',index=False)

In [132]:
dedup_df.to_csv('data/snooker_outcomes_dedup.csv',index=False)